# Tarea 1 Modelamiento y Optimización IN3171-1
# Integrantes (sec 1): Adolfo Rojas Valenzuela, Diego E. Cristallini
# Fecha de entrega: 01 de abril de 2024

# Parte a)

In [2]:
#Se instala la extensión de Gurobi
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 34.0 MB/s eta 0:00:00


In [3]:
from gurobipy import *
import numpy as np
import time

trabajadores = range(5) #Se define la cantidad de trabajadores
estaciones = range(5) #Se define la cantidad de estaciones
clientes = range(3) #Se define la cantidad de clientes

# Cantidad de producto pedido por el cliente k
demanda = [250, 150, 120]

# Matríz de productividad asociado a lo producido por el trabajador i si fuese contratado en la estación j
S = [[100, 180, 150,  90, 0],
    [80,   120,  90, 100, 0],
    [130,   80,  90,  85, 0],
    [200,   80, 120, 150, 0],
    [70,   130,  80, 130, 0]]

# Matríz de costos asociados a transportar producto desde la estación j al cliente k
C = [[10,  5,   5],
    [8,    5,   6],
    [8,    7,  10],
    [6,    3,   4],
    [0,    0,   0]]

# Parte b)

In [5]:
# Crear el modelo
modelov1 = Model("Optitalia v1")

## Variables de decisión:
w = modelov1.addVars(trabajadores, estaciones, vtype='B', name="w")
x = modelov1.addVars(estaciones, clientes, lb=0, vtype='I', name="x")

## Restricciones:

# Asignación:
modelov1.addConstrs((w.sum('*', j) == 1 for j in estaciones))                   # Cada estación debe tener 1 solo trabajador (la igualdad puede ser relajada, siempre será más óptimo tener las estaciones ocupadas)
modelov1.addConstrs((w.sum(i, '*') == 1 for i in trabajadores))                 # Cada trabajador solo puede ser asignado a 1 única estación (en caso de trabajar con 4 estaciones se necesita trabajar con <= pues no hay relación 1 a 1 posible)

# Demanda:
modelov1.addConstrs((x.sum('*', k) >= demanda[k] for k in clientes))            # La cantidad enviada a cada cliente debe ser a lo menos lo que este exige (de todas formas nunca se enviará más pues esto tiene un costo asociado)

# Relación entre variables:
for j in estaciones:
    modelov1.addConstr((quicksum(x[j,k] for k in clientes)) <= quicksum(w[i,j]*S[i][j] for i in trabajadores))    # Como no hay inventario previo, lo que vende/produce cada estación no puede ser más de lo que el trabajador contratado en dicha estación podría producir

## Función objetivo:
FO1 = quicksum(x[j,k]*C[j][k] for j in estaciones for k in clientes)
modelov1.setObjective(FO1, GRB.MINIMIZE)

# Medir el tiempo de ejecución antes de optimizar
start_time = time.time()

# Ocultar el texto y optimizar el modelo
modelov1.Params.LogToConsole = 0
modelov1.optimize()

# Medir el tiempo de ejecución después de optimizar
end_time = time.time()

# Verificar si se encontró una solución óptima
if modelov1.status == GRB.OPTIMAL:
    print("\nSolución óptima encontrada:")
    print()
    for i in trabajadores:
        for j in estaciones:
            if w[i, j].x == 1:
                if j+1 != 5:
                    print(f"Trabajador {i+1} fue asignado a la estación {j+1}")
                else:
                    print(f"Trabajador {i+1} no fue contratado")
    print()
    for k in clientes:
        for j in estaciones:
            if x[j, k].x > 0:
                print(f"Cantidad enviada desde estación {j+1} al cliente {k+1}: {int(x[j, k].X)}")

    # Imprimir el costo total
    costo_total = modelov1.objVal
    print(f"\nCosto total de la solución óptima: {costo_total}")

else:
    print("No se encontró solución óptima.")

# Calcular y mostrar el tiempo de ejecución
print(f"\nTiempo de ejecución: {end_time - start_time} segundos.")


Solución óptima encontrada:

Trabajador 1 fue asignado a la estación 3
Trabajador 2 no fue contratado
Trabajador 3 fue asignado a la estación 1
Trabajador 4 fue asignado a la estación 4
Trabajador 5 fue asignado a la estación 2

Cantidad enviada desde estación 2 al cliente 1: 100
Cantidad enviada desde estación 3 al cliente 1: 150
Cantidad enviada desde estación 4 al cliente 2: 150
Cantidad enviada desde estación 1 al cliente 3: 120

Costo total de la solución óptima: 3050.0

Tiempo de ejecución: 0.0033996105194091797 segundos.


# Parte c)

In [ ]:
# Matrices de producción y costo
P = [130, 180, 120, 100, 0]  # Ajuste para que coincida con el número de estaciones

# Crear el modelo
modelocf = Model("Optitalia Capacidades Fijas")

# Variables de decisión
x = modelocf.addVars(estaciones, clientes, lb=0, vtype='I', name="x")

# Restricciones de demanda
modelocf.addConstrs((x.sum('*', k) >= demanda[k] for k in clientes))

# Restricciones de capacidades fijas
for j in estaciones:
    modelocf.addConstr(x.sum(j, '*') <= P[j])

# Función objetivo
FO2 = quicksum(x[j, k] * C[j][k] for j in estaciones for k in clientes)
modelocf.setObjective(FO2, GRB.MINIMIZE)

# Medir el tiempo de ejecución antes de optimizar
start_time = time.time()

# Ocultar el texto y optimizar el modelo
modelocf.Params.LogToConsole = 0
modelocf.optimize()

# Medir el tiempo de ejecución después de optimizar
end_time = time.time()

# Mostrar la solución y el costo
if modelocf.status == GRB.OPTIMAL:
    print("\nSolución óptima encontrada:")
    print()
    for j in estaciones:
        for k in clientes:
            if x[j, k].x > 0:
                print(f"La cantidad enviada desde estación {j+1} al cliente {k+1}: {int(x[j, k].X)}")
    print(f"\nCosto total de la solución óptima: {modelocf.ObjVal}")
    print(f"Diferencia de costo Optitalia Capacidades Fijas y Optitalia v1: {abs(modelov1.ObjVal-modelocf.ObjVal)}")
else:
    print("No se encontró solución óptima.")

# Calcular y mostrar el tiempo de ejecución
print(f"\nTiempo de ejecución: {end_time - start_time} segundos.")


Solución óptima encontrada:

La cantidad enviada desde estación 1 al cliente 2: 10
La cantidad enviada desde estación 1 al cliente 3: 120
La cantidad enviada desde estación 2 al cliente 1: 130
La cantidad enviada desde estación 2 al cliente 2: 40
La cantidad enviada desde estación 3 al cliente 1: 120
La cantidad enviada desde estación 4 al cliente 2: 100

Costo total de la solución óptima: 3150.0
Diferencia de costo Optitalia Capacidades Fijas y Optitalia v1: 100.0

Tiempo de ejecución: 0.001965761184692383 segundos.


# Parte d)

In [ ]:
# Crear el modelo
modelo = Model("Optitalia v2")

## Variables de decisión:
w = modelo.addVars(trabajadores, estaciones, vtype='B', name="w")
x = modelo.addVars(estaciones, clientes, lb=0, vtype='I', name="x")
u = modelo.addVar(vtype='B', name="u")                                          # Esta es la variable binaria auxiliar que "detecta" si trabajadores 1 y 4 fueron asignados a la vez

## Restricciones:

# Asignación:
modelo.addConstrs((w.sum('*', j) == 1 for j in estaciones))
modelo.addConstrs((w.sum(i, '*') == 1 for i in trabajadores))

# Demanda:
modelo.addConstrs((x.sum('*', k) >= demanda[k] for k in clientes))

# Relación entre variables:
chambea1 = quicksum(w[1-1,j] for j in estaciones[:-1])                          # El [:-1] es para no contar la 5ta estación de desempleo
chambea4 = quicksum(w[4-1,j] for j in estaciones[:-1])

modelo.addConstr(u <= chambea1)                                                 # Si 1 no trabaja, no nos preocupamos. Caso contrario faltaría ver al trabajador 4
modelo.addConstr(u <= chambea4)                                                 # Análogo a la restricción anterior
modelo.addConstr(u >= chambea1 + chambea4 - 1)                                  # En caso de que ambos estén asignados, obligar a la variable que se active

for j in estaciones:
  if j+1 != 5:
    modelo.addConstr((quicksum(x[j,k] for k in clientes)) <= quicksum(w[i,j]*S[i][j] for i in trabajadores) - 10*u) # Por cada iteración restamos 10 de producción si u es 1
  # En el caso de que estemos en la estación de desempleo y los trabajadores 1, 4 estén asignados, obligamos a que la venta en el resto de estaciones no se vea afectada
  else:
    modelo.addConstr((quicksum(x[j,k] for k in clientes)) == 0)

## Función objetivo:
FO3 = quicksum(x[j,k]*C[j][k] for j in estaciones for k in clientes)
modelo.Params.LogToConsole = 0
modelo.setObjective(FO3, GRB.MINIMIZE)
modelo.optimize()

# Medir el tiempo de ejecución antes de optimizar
start_time = time.time()

if modelo.status == GRB.OPTIMAL:
  print(f"Solución óptima encontrada:")
  print()
  for i in trabajadores:
    for j in estaciones:
      if w[i, j].x == 1:
        if j+1 != 5:
          print(f"Trabajador {i+1} fue asignado a la estación {j+1}")
        else:
          print(f"Trabajador {i+1} no fue contratado")
        break
  print()
  for k in clientes:
    for j in estaciones:
      if x[j, k].x > 0:
        print(f"Cantidad enviada desde estación {j+1} al cliente {k+1}: {int(x[j, k].x)}")
  print(f"\nCosto total de la solución óptima: {modelo.ObjVal}")
else:
  print("No se encontró solución óptima.")

# Medir el tiempo de ejecución después de optimizar
end_time = time.time()

# Calcular y mostrar el tiempo de ejecución
print(f"\nTiempo de ejecución: {end_time - start_time} segundos.")

Solución óptima encontrada:

Trabajador 1 fue asignado a la estación 3
Trabajador 2 no fue contratado
Trabajador 3 fue asignado a la estación 1
Trabajador 4 fue asignado a la estación 4
Trabajador 5 fue asignado a la estación 2

Cantidad enviada desde estación 2 al cliente 1: 110
Cantidad enviada desde estación 3 al cliente 1: 140
Cantidad enviada desde estación 2 al cliente 2: 10
Cantidad enviada desde estación 4 al cliente 2: 140
Cantidad enviada desde estación 1 al cliente 3: 120

Costo total de la solución óptima: 3070.0

Tiempo de ejecución: 0.011181116104125977 segundos.
